<a href="https://colab.research.google.com/github/giambono/divine_semantics/blob/main/notebooks/run_compute_qdrant_embeddiing_optim_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Check if the notebook is running on Google Colab
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Clone the repository if running on Colab
if is_colab():
    print("Running on Google Colab. Cloning repository...")
    !git clone https://github.com/giambono/divine_semantics.git
    os.chdir("/content/divine_semantics")
    !pip install -r requirements.txt
else:
    import sys
    sys.path.append("..")
    print(f"Working directory set to: {os.getcwd()}")
    print("Not running on Google Colab.")

Running on Google Colab. Cloning repository...
Cloning into 'divine_semantics'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 147 (delta 65), reused 108 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 1.60 MiB | 10.06 MiB/s, done.
Resolving deltas: 100% (65/65), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

load_dotenv()

qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key,
)

In [3]:
from src.utils import load_model
from src.compute_embeddings import weighted_avg_embedding_qdrant


In [5]:
from qdrant_client.models import Distance, VectorParams

collection_name = "dante_multilingual_e5"
model_name = "multilingual_e5"
model = load_model(model_name)

sample_embedding = model.encode("test")
embedding_dim = len(sample_embedding)

# Example usage:
weight_map = {3: 0.45266643204217655, 4: 0.10424393059867922, 5: 0.44308963735914425}
# {'musa': 0.45266643204217655, 'kirkpatrick': 0.10424393059867922, 'durling': 0.44308963735914425}

qdrant_client.create_collection(
            collection_name=f"{collection_name}_optim_weights",
            vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
        )

df_weighted = weighted_avg_embedding_qdrant(
    model_name=model_name,
    collection_name=collection_name,
    qdrant_client=qdrant_client,  # Your pre-initialized QdrantClient instance
    author_weights=weight_map,
    batch_size=1000,
    store_in_qdrant=True,
    upsert_batch_size=200,
    collection_name_weighted=f"{collection_name}_optim_weights"
)
print(df_weighted)


      cantica_id  canto  start_verse  end_verse  \
0              1      1            1          3   
1              1      1            4          6   
2              1      1            7          9   
3              1      1           10         12   
4              1      1           13         15   
...          ...    ...          ...        ...   
1591           1     34          127        129   
1592           1     34          130        132   
1593           1     34          133        135   
1594           1     34          136        138   
1595           1     34          139        139   

                                   model  type_id author_id  \
0     weighted_embedding_multilingual_e5        1      None   
1     weighted_embedding_multilingual_e5        1      None   
2     weighted_embedding_multilingual_e5        1      None   
3     weighted_embedding_multilingual_e5        1      None   
4     weighted_embedding_multilingual_e5        1      None   
...      